In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

batch_size = 128
epochs = 15
IMG_HEIGHT = 60
IMG_WIDTH = 30
CLASS_NAMES = np.array([])
for i in range(10):
    CLASS_NAMES =np.append(CLASS_NAMES,chr(ord("0")+i))
for i in range(26):
    CLASS_NAMES = np.append(CLASS_NAMES,chr(ord("A")+i))

#training dir / validation dir
train_dir_data01 = os.path.abspath(os.getcwd())+"\\train\\data01_train\\"
val_dir_data01 = os.path.abspath(os.getcwd())+"\\dev\\data01_dev"
num_train_data01 = len(os.listdir(train_dir_data01))
num_val_data01 = len(os.listdir(val_dir_data01))
print("Train data count: " + str(num_train_data01))
print("Validation data count: " + str(num_val_data01))

#store filename and label to a Dataframe 
import pandas as pd
data = pd.read_csv("train\\data01_train.csv")
for i in range(len(data)):
    data.iloc[i,1] = list(data.iloc[i,1])

#maybe there's an easier way to separate labels??
arr = np.zeros([6,50000],str)
for j in range(6):
    for i in range(len(data)):
        arr[j][i] = data.iloc[i,1][j]
data = data.drop(columns= ["code"]).join(pd.DataFrame(arr.transpose(),columns = ["code0","code1",'code2','code3','code4','code5']))

#for testing model
data = data.drop(columns=["code1","code2","code3","code4","code5"])
data

Train data count: 50000
Validation data count: 10000


,filename,code0
0,000000.jpg,Z
1,000001.jpg,V
2,000002.jpg,J
3,000003.jpg,X
4,000004.jpg,H
...,...,...
49995,049995.jpg,3
49996,049996.jpg,4
49997,049997.jpg,C
49998,049998.jpg,K


In [2]:
label = tf.data.Dataset.from_tensor_slices(data.iloc[:,1])
def parse_label(strIN):
    return tf.expand_dims(strIN==CLASS_NAMES,0)

def cropimage_operation(fn):
    a = tf.io.read_file((fn))
        #crop to [60,30,3]
    img = tf.image.decode_and_crop_jpeg(a,[0,0,IMG_HEIGHT,IMG_WIDTH],channels=3)
    return tf.expand_dims(img,0)

train_label01 = label.map(parse_label)
train_dir01 = tf.data.Dataset.list_files(train_dir_data01+'*.jpg',shuffle=False)
train_data01 = tf.data.Dataset.zip((train_dir01.map(cropimage_operation),train_label01))

def show_batch(image_batchh, label_batchh):
  plt.figure(figsize=(14,10))
  for n in range(25):
      ax = plt.subplot(5,5,n+1)
      plt.imshow(image_batchh[n][0])
      plt.title(CLASS_NAMES[np.where(label_batch[n][0])])
      plt.axis('image')



In [9]:
model = Sequential([
    Conv2D(64, (9, 9), input_shape=(IMG_HEIGHT, IMG_WIDTH ,3), padding='same',
           activation='relu'),
    MaxPooling2D(pool_size=(2,2),strides=2),
    BatchNormalization(axis=1),
    Conv2D(64, (9, 9), activation='relu', padding='same'),     
    BatchNormalization(axis=1),
    MaxPooling2D(pool_size=(2,2),strides=2),
    Conv2D(64, (7, 7), activation='relu', padding='same'),
    Conv2D(32, (7, 7), activation='relu', padding='same',),   
    Flatten(),
    Dense(36,activation="softmax")
])
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 60, 30, 64)        15616     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 15, 64)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 15, 64)        120       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 15, 64)        331840    
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 15, 64)        120       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 7, 64)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 7, 64)         2

In [13]:
history = model.fit(
    train_data01,
    batch_size=batch_size,
    steps_per_epoch=50000// batch_size, 
    epochs=epochs,
    verbose=1
)

Epoch 1/15
390/390 [==============================] - 2s 6ms/step - loss: 3.5838 - accuracy: 0.0256
Epoch 2/15
390/390 [==============================] - 2s 6ms/step - loss: 3.5803 - accuracy: 0.0462
Epoch 3/15
390/390 [==============================] - 2s 6ms/step - loss: 3.5809 - accuracy: 0.0308
Epoch 4/15
390/390 [==============================] - 2s 6ms/step - loss: 3.5841 - accuracy: 0.0256
Epoch 5/15
390/390 [==============================] - 2s 6ms/step - loss: 3.5815 - accuracy: 0.0308
Epoch 6/15
390/390 [==============================] - 2s 6ms/step - loss: 3.5853 - accuracy: 0.0359
Epoch 7/15
390/390 [==============================] - 2s 6ms/step - loss: 3.5821 - accuracy: 0.0282
Epoch 8/15
390/390 [==============================] - 2s 6ms/step - loss: 3.5830 - accuracy: 0.0231
Epoch 9/15
390/390 [==============================] - 2s 6ms/step - loss: 3.5858 - accuracy: 0.0231
Epoch 10/15
390/390 [==============================] - 2s 6ms/step - loss: 3.5817 - accuracy: 0.0282

In [12]:
model.optimizer.lr=0.0001

In [ ]:
def predict_sigle_img(imgdir):
    imgtest2 = plt.imread(imgdir)
    imgtest2 = np.expand_dims(imgtest2, axis=0)
    result = model(imgtest2)
    PredChr = CLASS_NAMES[
        np.where(result.numpy()[0] == result.numpy()[0].max())[0][0]
    ]
    print("Predict character: "+ PredChr)
    return model(imgtest2)